# Greedy Policy

In this notebook, we demonstrate using a simple greedy policy for [Citi Bike](https://maro.readthedocs.io/en/latest/scenarios/citi_bike.html), a real-world bike repositioning scenario. Our greedy policy is simple: if the event type is supply, the policy will make the current station send as many bikes as possible to one of k stations with the most empty docks. If the event type is demand, the policy will make the current station request as many bikes as possible from one of k stations with the most bikes. We use a heap data structure to find the top k supply/demand candidates from the action scope associated with each decision event.

In [1]:
import heapq
import random

from maro.simulator import Env
from maro.simulator.scenarios.citi_bike.common import Action, DecisionEvent, DecisionType


class GreedyPolicy:
    def __init__(self, supply_top_k: int = 1, demand_top_k: int = 1):
        self._supply_top_k = supply_top_k
        self._demand_top_k = demand_top_k

    def choose_action(self, decision_event: DecisionEvent):
        if decision_event.type == DecisionType.Supply:
            # Find k target stations with the most empty slots, randomly choose one of them and send as many bikes to
            # it as allowed by the action scope.
            top_k_demands = []
            for demand_candidate, available_docks in decision_event.action_scope.items():
                if demand_candidate == decision_event.station_idx:
                    continue

                heapq.heappush(top_k_demands, (available_docks, demand_candidate))
                if len(top_k_demands) > self._demand_top_k:
                    heapq.heappop(top_k_demands)

            max_reposition, target_station_idx = random.choice(top_k_demands)
            action = Action(decision_event.station_idx, target_station_idx, max_reposition)
        else:
            # Find k source stations with the most bikes, randomly choose one of them and request as many bikes from
            # it as allowed by the action scope.
            top_k_supplies = []
            for supply_candidate, available_bikes in decision_event.action_scope.items():
                if supply_candidate == decision_event.station_idx:
                    continue

                heapq.heappush(top_k_supplies, (available_bikes, supply_candidate))
                if len(top_k_supplies) > self._supply_top_k:
                    heapq.heappop(top_k_supplies)

            max_reposition, source_idx = random.choice(top_k_supplies)
            action = Action(source_idx, decision_event.station_idx, max_reposition)

        return action

# Interaction with the Greedy Policy

This environment is driven by [real trip history data](https://s3.amazonaws.com/tripdata/index.html) from Citi Bike. 

In [2]:
env = Env(scenario="citi_bike", topology="ny.201801", start_tick=0, durations=2880, snapshot_resolution=10)
policy = GreedyPolicy()
metrics, decision_event, done = env.step(None)
while not done:
    metrics, decision_event, done = env.step(policy.choose_action(decision_event))

print(f"Greedy policy performance: {env.metrics}")
env.reset()

Greedy policy performance: {'trip_requirements': 17729, 'bike_shortage': 12305, 'operation_number': 229229}
